In [1]:
import json, random, re, collections, itertools, base64, hashlib
from dataclasses import dataclass
from copy import deepcopy
from pathlib import Path
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from operator import itemgetter as at
from IPython.core.display import display, HTML
from ipywidgets import interact
display_html = lambda x: display(HTML(x))
from pprint import pprint as pp
ls = lambda path: list(map(str, path.iterdir()))
annot_path = Path("../annotations")
data_path = Path("../data")
preprocessed_path = Path("../preprocessed")

## Read mapping

In [2]:
with (data_path/"annotaions.json").open('r') as f:
    annotations = json.load(f)

In [3]:
with (preprocessed_path/"resources.json").open('r') as f:
    resources = json.load(f)
with (preprocessed_path/"ingredients.json").open('r') as f:
    ingredients = json.load(f)
with (preprocessed_path/"labels.json").open('r') as f:
    idx2label = [tuple(t) for t in json.load(f)]
    label2idx={r:i for i,r in enumerate(idx2label)}
ingredient_dict = {k:v for k,v in ingredients}
resource_dict = {k:v for k,v in resources}

In [4]:
ls(preprocessed_path)

['../preprocessed/103308.npz',
 '../preprocessed/ingredients.json',
 '../preprocessed/resources.json',
 '../preprocessed/labels.json',
 '../preprocessed/103308.json']

In [5]:
states = np.load(str(preprocessed_path/ "103308.npz" ))
states.files

['103308_0',
 '103308_1',
 '103308_2',
 '103308_3',
 '103308_4',
 '103308_5',
 '103308_6',
 '103308_7',
 '103308_8',
 '103308_9']

In [6]:
states['103308_3']

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [7]:
annotations["103308"]

[{'0': [['A1', 'I10_d4pRP'],
   ['A1', 'I7oUHHY41'],
   ['A1', 'IN0CxTNVh'],
   ['A1', 'INNbMITPe'],
   ['A1', 'IW2FkWnJk'],
   ['A1', 'Itg64O_Uz'],
   ['A1', 'IukXkN8DV'],
   ['A1', 'Iynxt1P5R'],
   ['A1', 'LIMMEDIATE']]},
 {'1': [['A1', 'IN0CxTNVh'],
   ['A1', 'INNbMITPe'],
   ['A1', 'IW2FkWnJk'],
   ['A1', 'Itg64O_Uz'],
   ['A1', 'IukXkN8DV'],
   ['A1', 'Iynxt1P5R'],
   ['A1', 'LIMMEDIATE'],
   ['GW4', 'I10_d4pRP'],
   ['GW4', 'I7oUHHY41'],
   ['GW4', 'LIMMEDIATE'],
   ['GW4', 'TSKILLET']],
  '0': [['A1', 'I10_d4pRP'],
   ['A1', 'I7oUHHY41'],
   ['A1', 'IN0CxTNVh'],
   ['A1', 'INNbMITPe'],
   ['A1', 'IW2FkWnJk'],
   ['A1', 'Itg64O_Uz'],
   ['A1', 'IukXkN8DV'],
   ['A1', 'Iynxt1P5R'],
   ['A1', 'LIMMEDIATE']]},
 {'1': [['A1', 'IN0CxTNVh'],
   ['A1', 'INNbMITPe'],
   ['A1', 'IW2FkWnJk'],
   ['A1', 'Itg64O_Uz'],
   ['A1', 'LIMMEDIATE'],
   ['GW4', 'I10_d4pRP'],
   ['GW4', 'I7oUHHY41'],
   ['GW4', 'LIMMEDIATE'],
   ['GW4', 'TSKILLET'],
   ['W1', 'IukXkN8DV'],
   ['W1', 'Iynxt1P5R'],
   

In [8]:
@dataclass
class Instruction:
    ts: int
    command: str
    ingredient: str
    resource:str

In [23]:
def program(annotations):
    return sum([[v for k,v in a.items() if k!='0' and v[0]!='A1'] for a in annotations],[])
    
len (program(annotations["103308"]))

12

In [50]:
def program_step(annotation):
    max_ts = max(map(int, annotation.keys()))
    new_state = None
    state = {res: set() for res in resource_dict}
    for resource, ing in annotation['0']: 
        state[resource].add(ing)
    actions = []
    for ts in range(1, 1+max_ts):
        new_state = {res: set() for res in resource_dict}
        for resource, ing in annotation[str(ts)]: 
            new_state[resource].add(ing)
        for resource in new_state.keys():
            added_ings = new_state[resource]-state[resource]
            removed_ings = state[resource]-new_state[resource]
            for ing in added_ings:
                if ing.startswith("L"):
                    if ing!="LIMMEDIATE":
                        actions.append(Instruction(ts, "chef_check", ing, resource))
                elif ing.startswith("T"):
                    actions.append(Instruction(ts, "use", ing, resource))
                else:
                    if resource=="A1":
                        pass
                    elif resource=="A0":
                        actions.append(Instruction(ts, "discard", ing, ""))
                    else:
                        actions.append(Instruction(ts, "put", ing, resource))
            for ing in removed_ings:
                if not ing.startswith("L") and not resource.startswith('A'):
                    actions.append(Instruction(ts, "remove", ing, resource))
        state = deepcopy(new_state)
    for res in resource_dict:
        for ts in range(1, 1+max_ts):
            before = {a.ingredient for a in actions if a.ts<ts and a.command=="put" and a.resource==res and a.ingredient.startswith("I")}
            after = {a.ingredient for a in actions if a.ts==ts and a.command=="remove" and a.resource==res and a.ingredient.startswith("I")}
            new_res = {a.resource for a in actions if a.ts==ts and a.command=="put" and a.resource!=res and a.ingredient in {b for b in before}}
            if len(new_res)!=1:
                continue
            new_res=list(new_res)[0]
            if before<=after:
                actions.append(Instruction(ts, "move_contents", res, new_res))
                actions = [a for a in actions if not (
                    (a.ts==ts and (a.command,a.resource) in {("remove", res), ("put", new_res)} and a.ingredient in before)
                )]
                    
    action_order=["move_contents", "remove", "use", "put", "chef_check", "discard", "serve"]
    actions = sorted(actions, key=lambda t: (t.ts,action_order.index(t.command)))
    return actions

def program(annotation, verboze=False):
    """Runs program_step for each item in list, and fix timestamps"""
    actions = []
    ts=0
    for a in annotation:
        p = program_step(a)
        if len(p)==0:
            continue
        for instruction in p:
            instruction.ts+=ts
        actions.extend(p)
        ts=p[-1].ts
    if verboze:
        actions = [(a.ts,a.command,ingredient_dict.get(a.ingredient, resource_dict.get(a.ingredient, "")),resource_dict.get(a.resource)) for a in actions]
    return actions 
pp (program(annotations["103308"], verboze=True))

[(1, 'use', 'Skillet', 'Stove/Medium-High'),
 (1, 'put', 'butter', 'Stove/Medium-High'),
 (1, 'put', 'vegetable oil', 'Stove/Medium-High'),
 (2, 'use', 'Skillet', 'Stove/Medium-High'),
 (2, 'put', 'salt and ground black pepper to taste', 'Countertop/Combine/Mix'),
 (2, 'put', 'boneless pork chops', 'Countertop/Combine/Mix'),
 (2, 'put', 'butter', 'Stove/Medium-High'),
 (2, 'put', 'vegetable oil', 'Stove/Medium-High'),
 (3, 'move_contents', 'Countertop/Combine/Mix', 'Countertop/Coat/Sprinkle'),
 (3, 'put', 'all-purpose flour', 'Countertop/Coat/Sprinkle'),
 (4, 'use', 'Skillet', 'Stove/Medium-High'),
 (4, 'put', 'butter', 'Stove/Medium-High'),
 (4, 'put', 'boneless pork chops', 'Stove/Medium-High'),
 (4, 'put', 'salt and ground black pepper to taste', 'Stove/Medium-High'),
 (4, 'put', 'vegetable oil', 'Stove/Medium-High'),
 (4, 'put', 'all-purpose flour', 'Stove/Medium-High'),
 (4, 'chef_check', 'For X minutes', 'Stove/Medium-High'),
 (5, 'use', 'Other', 'Oven/Medium'),
 (5, 'put', 'butt